In [1]:
import os
import uuid

def short_filename(filepath):
    # Split the filepath into a directory and filename
    dir_path, filename = os.path.split(filepath)
    
    # Generate a unique filename using uuid
    new_filename = str(uuid.uuid4())
    
    # Preserve the file extension
    extension = os.path.splitext(filepath)[1]
    new_filepath = os.path.join(dir_path, new_filename + extension)

    # Rename the file
    # os.rename(filepath, new_filepath)
    
    print(f'File renamed to: {new_filepath}')
    
    return new_filepath

In [2]:
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

# Define root domain to crawl
domain = "www.experian.com"
full_url = "https://www.experian.com/"

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# Function to get the hyperlinks from a URL
def get_hyperlinks(url):
    
    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []
            
            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))


def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        filename = 'text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt"
        filename_len_limit = 200
        if len(filename) > filename_len_limit:
            filename = short_filename(filename)

        num_failed = 0
        try:
            with open(filename, "w") as f:
    
                # Get the text from the URL using BeautifulSoup
                soup = BeautifulSoup(requests.get(url).text, "html.parser")
    
                # Get the text but remove the tags
                text = soup.get_text()
    
                # If the crawler gets to a page that requires JavaScript, it will stop the crawl
                if ("You need to enable JavaScript to run this app." in text):
                    print("Unable to parse page " + url + " due to JavaScript being required")
                
                # Otherwise, write the text to the file in the text directory
                f.write(text)
        except:
            num_failed += 1
            print("Failed to download: " + str(num_failed))
            print("Failed url: " + str(url))

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://www.experian.com/
https://www.experian.com/blogs/ask-experian/best-credit-card-benefits
https://www.experian.com/blogs/ask-experian/category/credit-cards/secured-credit-cards
https://www.experian.com/blogs/ask-experian/what-is-a-starter-credit-card-and-how-can-it-help-build-credit
https://www.experian.com/blogs/ask-experian/best-secured-credit-cards
https://www.experian.com/blogs/ask-experian/are-credit-cards-safer-than-debit-cards
https://www.experian.com/blogs/ask-experian/credit-card-or-debit-card-to-improve-credit-score
https://www.experian.com/blogs/ask-experian/credit-education/score-basics/700-credit-score
https://www.experian.com/blogs/ask-experian/how-utilization-rate-affects-credit-scores
https://www.experian.com/blogs/ask-experian/balance-on-report-is-usually-what-appears-on-statement
https://www.experian.com/blogs/ask-experian/better-pay-off-credit-card-full-every-month-or-maintain-balance
https://www.experian.com/blogs/ask-experian/how-compound-interest-works
https

In [3]:
def remove_newlines(serie):
    serie = serie.str.replace('\n', ' ')
    serie = serie.str.replace('\\n', ' ')
    serie = serie.str.replace('  ', ' ')
    serie = serie.str.replace('  ', ' ')
    return serie

In [4]:
import pandas as pd

# Create a list to store the text files
texts=[]

# Get all the text files in the text directory
for file in os.listdir("text/" + domain + "/"):

    # Open the file and read the text
    with open("text/" + domain + "/" + file, "r") as f:
        text = f.read()

        # Omit the first 11 lines and the last 4 lines, then replace -, _, and #update with spaces.
        texts.append((file[11:-4].replace('-',' ').replace('_', ' ').replace('#update',''), text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns = ['fname', 'text'])

# Set the text column to be the raw text with the newlines removed
df['text'] = df.fname + ". " + remove_newlines(df.text)
df.to_csv('processed/scraped.csv')
df.head()

,fname,text
0,n.com content dam marketing na healthcare broc...,n.com content dam marketing na healthcare broc...
1,n.com small business business public records,n.com small business business public records. ...
2,n.com blogs ask experian which type of savings...,n.com blogs ask experian which type of savings...
3,n.com blogs global insights category technolog...,n.com blogs global insights category technolog...
4,n.com blogs ask experian medical bill payment ...,n.com blogs ask experian medical bill payment ...


In [ ]:
with open('processed/scraped.csv', 'r') as f_in, open('processed/scraped_null_removed.csv', 'w') as f_out:
    for line in f_in:
        clean_line = line.replace('\0', '')  # or replace with space if you prefer
        f_out.write(clean_line)